# Visualize the embedding space

Contains code to generate embeddings for the test data as well as plot and save to a checkpoint file which can be plotting using tensorboard

In [ ]:
import tensorflow as tf
from model.seq2seq import bilstm, lstm, gru
import numpy as np
from utils.text_processing import load_dict_from_vocab_file

vocab_file = './data/character_inventory_unk.txt'
traindb_file = './data/training.npz'
testdb_file = './data/testing.npz'
checkpoint_file = './tfmodel/gru_enc/model_800.tfmodel'

## Get the embeddings

In [ ]:
vocab = load_dict_from_vocab_file(vocab_file)
vocab_size = len(vocab)
lstm_dim = 500

In [ ]:
with tf.name_scope("placeholders"):
    encoder_in = tf.placeholder(tf.int32, [None, None])
    encoder_lens = tf.placeholder(tf.int32, [None])
    batch_size = tf.placeholder(tf.int32)

with tf.name_scope("model"):
    enc_emb = tf.one_hot(encoder_in, vocab_size)
    with tf.name_scope("encoder"):
        encoder_state = gru(enc_emb, encoder_lens, lstm_dim, batch_size=batch_size, scope='gru')
        
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, checkpoint_file)

In [ ]:
data = np.load(testdb_file)
encoder_in_batch = data['enc_in']
encoder_len_batch = data['enc_lens']

embeddings = sess.run(encoder_state, feed_dict={encoder_in: encoder_in_batch, 
                                                   encoder_lens: encoder_len_batch,
                                                   batch_size: encoder_in_batch.shape[0]})

## PCA and plotting

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mpld3

%matplotlib notebook
mpld3.enable_notebook()

def scatterPlotWithLabels(ax,x,y,z=[], pointSize=5,labels=None,color='red'):
    if len(z) > 0:
        handle = ax.scatter(x, y, z, c=color, s=pointSize)
    else:
        handle = ax.scatter(x, y, c=color, s=pointSize)
    
    if labels is not None:
        tooltip = mpld3.plugins.PointLabelTooltip(handle, labels=labels)
        mpld3.plugins.connect(plt.gcf(), tooltip)
    return handle

def get_cmap(n, name='RdYlBu'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

def plotSamples(classes, xform, classnames, projection=None, title=None): 
    colormap = get_cmap(len(classes))
    fig = plt.figure(figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
    fig.subplots_adjust(right=0.8)
    ax = plt.subplot(facecolor='#EEEEEE', projection=projection)
    #ax.grid(color='white', linestyle='solid')
    if title is not None:
        ax.set_title(title, fontsize=30)
    handles = []
    labels = []
    point_labels = []
    for i,c in enumerate(classes):
        plot_data = xform.transform(c)
        if projection and projection in '3d':
            ex = scatterPlotWithLabels(ax, plot_data[:,0], plot_data[:,1], plot_data[:,2], color=colormap(i))
        else:
            ex = scatterPlotWithLabels(ax, plot_data[:,0], plot_data[:,1], color=colormap(i))

        handles.append(ex)
        labels.append(classnames[i])

#     handles, labels = ax.get_legend_handles_labels() # return lines and labels
#     interactive_legend = mpld3.plugins.InteractiveLegendPlugin(zip(handles,
#                                                              ax.collections),
#                                                          labels,
#                                                          #alpha_unsel=0.5,
#                                                          #alpha_over=1.5, 
#                                                          start_visible=True)
#     #ax.legend(handles, labels, markerscale=0.5)
#     mpld3.plugins.connect(fig, interactive_legend)

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

data = pd.read_csv('./data/testing.txt', sep='\t')
classes = [d.split(';')[0] for d in data.values[:,3]]
embed_pca = PCA(n_components=3)
embed_decomp = embed_pca.fit(embeddings)

In [ ]:
from itertools import groupby
classnames = [j for j,_ in groupby(classes)]
count_dups = [sum(1 for _ in group) for _, group in groupby(classes)]
embeddings_class_split = np.split(embeddings, np.cumsum(count_dups))[:-1]

plotSamples(embeddings_class_split, embed_decomp, classnames)

## Save embeddings as model checkpoint in order to plot in tensorboard

In [ ]:
from tensorflow.contrib.tensorboard.plugins import projector
import os

embedding_var = tf.Variable(embeddings, name='phylogenetic_tree')

# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = 'testing.tsv'

# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.summary.FileWriter('./tb/embeddings')

# The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
# read this file during startup.
projector.visualize_embeddings(summary_writer, config)

sess2 = tf.Session()
sess2.run(tf.global_variables_initializer())
saver2 = tf.train.Saver()
saver2.save(sess2, './tb/embeddings/model.ckpt', 0)